### Targeted information retrieval

We have seen how to parse a webpage, which retrieves the information without distinction.

But, in general, the purpose of scraping is to automate the collection of targeted information on the web.


In [1]:
from bs4 import BeautifulSoup
import requests

Let's say I want to scrape the description of the latest movies released in theaters.

So we'll go to the Allociné website and try to find the tags that will give me links to the specific pages of these movies to get their summaries.

#### Recovery of the url of the pages of films newly released in the theaters

In [2]:
url = "http://www.allocine.fr/"
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "lxml")
soup

http://www.allocine.fr/ 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>AlloCiné : Cinéma, Séries TV, BO de films et séries, Vidéos, DVD et VOD</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="AlloCiné, le site de référence du cinéma et des séries tv ! Découvrez notre recherche d'horaires de films, le programme tv de vos séries préférées, l'actualité ciné et séries, les émissions

On your web browser (Chrome, Firefox,...), you can use the "inspect" function (right click -> inspect) and drag your mouse to the areas of the page that interest you. At the same time you will see the part of the HTML script highlighted that corresponds to the area that interests you.

That's how you find the tags that you are interested in.

I notice that the relative link of the web page specific to the new movie on the poster is stored in these tags:

```html
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=235582.html" title="Le Grand Bain">Le Grand Bain</a>
```


In [3]:
for p in soup.find_all("a"):
    print(p.text)

Meilleurs films d'animation Disney
Meilleurs films Pixar

            Cinéma
        

            Séries
        

            Streaming
        

            Trailers
        

            VOD
        

            Les indés
        
séances cinéma
news
dossiers
émissions AlloCiné
dernières bandes-annonces
Agenda des sorties
Kids
Meilleurs films
Meilleurs documentaires
Avant-premières
Box Office
Films pour enfants à l'affiche
DVD
Meilleurs films 2019
Tous les films pour enfants
Tous les films
Sorties de la semaine
Mission : Impossible – The Final Reckoning
Lilo & Stitch
La Venue de l’avenir
Jeunes mères
Les Maudites
Ollie
Libertate
Ace
Trou salopard
Narivetta
Thunderbolts*
Partir un Jour
Les Musiciens
Destination Finale Bloodlines
Tous les films encore à l'affiche
 Un simple accident 
 Dragons 
 Alpha 
 28 ans plus tard 
 La Petite Dernière 
Tous les films à venir
Dernières news cinéma
C'est le meilleur démarrage au box-office de l'année et une suite pourrait déjà voir le jour !
C'es

This time, the site is more difficult to "extract". Let's use much more specific parameters to the search function `find_all`.

In [4]:
# In addition to the tag `a`, which is easily identifiable, we notice some additional
# information such as the value of the class variable of these identical tags.
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem)

<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=270936.html" title="Mission : Impossible – The Final Reckoning">Mission : Impossible – The Final Reckoning</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=268362.html" title="Lilo &amp; Stitch">Lilo &amp; Stitch</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=326174.html" title="La Venue de l’avenir">La Venue de l’avenir</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000005218.html" title="Jeunes mères">Jeunes mères</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=313568.html" title="Les Maudites">Les Maudites</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=316785.html" title="Ollie">Ollie</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=306892.html" title="Libertate">Libertate</a>
<a class="meta-title meta-title-link" href="/film/fichefilm_gen_cfilm=1000024169.ht

#### Recovery of `href`

We have noticed the presence of `href` links to the pages that interest us. Let's go get them back:

In [5]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("href"))

/film/fichefilm_gen_cfilm=270936.html
/film/fichefilm_gen_cfilm=268362.html
/film/fichefilm_gen_cfilm=326174.html
/film/fichefilm_gen_cfilm=1000005218.html
/film/fichefilm_gen_cfilm=313568.html
/film/fichefilm_gen_cfilm=316785.html
/film/fichefilm_gen_cfilm=306892.html
/film/fichefilm_gen_cfilm=1000024169.html
/film/fichefilm_gen_cfilm=1000023331.html
/film/fichefilm_gen_cfilm=1000022590.html
/film/fichefilm_gen_cfilm=297835.html
/film/fichefilm_gen_cfilm=1000001896.html
/film/fichefilm_gen_cfilm=1000015043.html
/film/fichefilm_gen_cfilm=197022.html
/series/ficheserie_gen_cserie=1000000466.html
/series/ficheserie_gen_cserie=36103.html
/series/ficheserie_gen_cserie=26316.html
/series/ficheserie_gen_cserie=36542.html
/series/ficheserie_gen_cserie=1000000164.html
/series/ficheserie_gen_cserie=31593.html
/series/ficheserie_gen_cserie=35813.html


Can you retrieve the titles for me by searching for `"title"` in the items of the previous list?

In [8]:
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    print(elem.get("title"))

Mission : Impossible – The Final Reckoning
Lilo & Stitch
La Venue de l’avenir
Jeunes mères
Les Maudites
Ollie
Libertate
Ace
Trou salopard
Narivetta
Thunderbolts*
Partir un Jour
Les Musiciens
Destination Finale Bloodlines
Secrets We Keep
Sirens
The Last Of Us
Rien ne t'efface
Tout pour la lumière
Ça : Bienvenue à Derry
Les Dossiers oubliés


#### Get summary

Let's start by building the URL that we will use to retrieve the summaries.

Start by putting the `href` values in a list of links.


In [9]:
links = []
for elem in soup.find_all("a", attrs={"class": "meta-title meta-title-link"}):
    links.append(elem.get("href"))
links

['/film/fichefilm_gen_cfilm=270936.html',
 '/film/fichefilm_gen_cfilm=268362.html',
 '/film/fichefilm_gen_cfilm=326174.html',
 '/film/fichefilm_gen_cfilm=1000005218.html',
 '/film/fichefilm_gen_cfilm=313568.html',
 '/film/fichefilm_gen_cfilm=316785.html',
 '/film/fichefilm_gen_cfilm=306892.html',
 '/film/fichefilm_gen_cfilm=1000024169.html',
 '/film/fichefilm_gen_cfilm=1000023331.html',
 '/film/fichefilm_gen_cfilm=1000022590.html',
 '/film/fichefilm_gen_cfilm=297835.html',
 '/film/fichefilm_gen_cfilm=1000001896.html',
 '/film/fichefilm_gen_cfilm=1000015043.html',
 '/film/fichefilm_gen_cfilm=197022.html',
 '/series/ficheserie_gen_cserie=1000000466.html',
 '/series/ficheserie_gen_cserie=36103.html',
 '/series/ficheserie_gen_cserie=26316.html',
 '/series/ficheserie_gen_cserie=36542.html',
 '/series/ficheserie_gen_cserie=1000000164.html',
 '/series/ficheserie_gen_cserie=31593.html',
 '/series/ficheserie_gen_cserie=35813.html']

The absolute URL of the movie pages we are looking for are built in this form: http://www.allocine.fr/film/fichefilm_gen_cfilm=243835.html

It is therefore necessary to take the previous list and build the absolute URLs for our search
It's up to you to play.

NB: Do not take the links for the shows(series). We only want movies.

In [10]:
movie_links = ["http://www.allocine.fr" + elem for elem in links if "film" in elem]
movie_links

['http://www.allocine.fr/film/fichefilm_gen_cfilm=270936.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=268362.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=326174.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000005218.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=313568.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=316785.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=306892.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000024169.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000023331.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000022590.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=297835.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000001896.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=1000015043.html',
 'http://www.allocine.fr/film/fichefilm_gen_cfilm=197022.html']

Finally, on each page, the title and synopsis must be retrieved. Let's try for a movie, the first of the list.

In [11]:
url = movie_links[0]
r = requests.get(url)
print(url, r.status_code)
soup = BeautifulSoup(r.content, "lxml")
soup

http://www.allocine.fr/film/fichefilm_gen_cfilm=270936.html 200


<!DOCTYPE html>
<html lang="fr">
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1,max-image-preview:standard" name="robots"/>
<title>Mission : Impossible – The Final Reckoning - Film 2025 - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Mission : Impossible – The Final Reckoning est un film réalisé par Christopher McQuarrie avec Tom Cruise, Hayley Atwell. Synopsis : Ce film est présenté en hors-compéti

For title 
```html
<div class="titlebar-title titlebar-title-lg">Le Grand Bain</div>
```
For the synopsis

```html
<div class="content-txt " itemprop="description"

 
              C’est dans les couloirs de leur piscine municipale que Bertrand, Marcus, Simon, Laurent, Thierry et les autres s’entraînent sous l’autorité toute relative de Delphine, ancienne gloire des bassins. Ensemble, ils se sentent libres et utiles. Ils vont mettre toute leur énergie dans une discipline jusque-là propriété de la gent féminine : la natation synchronisée. Alors, oui c’est une idée plutôt bizarre, mais ce défi leur permettra de trouver un sens à leur vie...
    
      </div>
```

In [38]:
for elem_tit in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-xl"}):
    # Just like that
    print(elem_tit.text)

for elem_syn in soup.find_all("p", attrs={"class": "bo-p"}):
    # Just like that
    print(elem_syn.text)

Destination Finale Bloodlines
Hantée par un cauchemar terrifiant qui revient sans cesse, Stefanie, étudiante à l’université, rentre chez elle pour retrouver la trace de la seule personne susceptible d’enrayer ce cycle infernal et de sauver ses proches du sort funeste qui les attend…


1) Automate this script for the entire list of movies.

2) Put the information in three lists (`film_links`, `title`, `synopsis`).

3) Create a `DataFrame` object from the `pandas` library that you will have to install. This dataframe will have to include these three informations in three columns.

4) Save this dataframe in a CSV file.

And here's your first real scrap, you're real hackers now.

In [40]:
import time
import random
from random import randint

titles = []
synopsises = []

# Here are the things you will have to do for all links:
# - Slow down the frequency of requests to avoid being identified and therefore banned from the website.
# Use `time.sleep(random.uniform(1.0, 2.0))`
# - Get request object from URL
# - Extract the content into a variable using BeautifulSoup
# - Get title
# - Get synopsis
for link in movie_links:
    req = requests.get(link)
    print(link, req.status_code)
    soup = BeautifulSoup(req.content, "lxml")
    for title in soup.find_all("div", attrs={"class": "titlebar-title titlebar-title-xl"}):
        titles.append(title.text)
    for synopsis in soup.find_all("p", attrs={"class": "bo-p"}):
        synopsises.append(synopsis.text)
        break
    time.sleep(random.uniform(1.0, 2.0))
# Check the length of the lists before creating the dataframe
len(titles), len(synopsises), len(movie_links)

http://www.allocine.fr/film/fichefilm_gen_cfilm=270936.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=268362.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=326174.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000005218.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=313568.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=316785.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=306892.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000024169.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000023331.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000022590.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=297835.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000001896.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=1000015043.html 200
http://www.allocine.fr/film/fichefilm_gen_cfilm=197022.html 200


(14, 14, 14)

In [41]:
import pandas as pd

df = pd.DataFrame({"Title": titles})
df["Synopsis"] = synopsises
df["Links"] = movie_links

In [42]:
df.to_csv("./assets/allo_cine.csv", index=False)